In [2]:
# imports
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy import signal
from astropy.io import fits
from astropy.visualization import astropy_mpl_style
import pickle
import math
from tqdm import tqdm
from scipy.signal import medfilt2d
    
plt.style.use(astropy_mpl_style)

In [31]:
# Preprocessing class

class Preprocessing:
    def __init__(self, fits_folder_paths, angle_folder, ccd_folder, background_ccd_folder, raw_angles_file_paths, angle_file, display_images_folder):
        '''
        initializes a Preprocessing object for this class. It 
        identifies the paths to the folders containing fits files, as 
        well as the names of the folders where the angles and ccd 
        numpy arrays will be kept.
        '''
        self.fits_folder_paths = fits_folder_paths
        self.angle_folder = angle_folder
        self.ccd_folder = ccd_folder
        self.raw_angles_file_paths = raw_angles_file_paths
        self.background_ccd_folder = background_ccd_folder
        self.angle_file = angle_file
        self.display_images_folder = display_images_folder

        # dictionary ffi->orbit for all images taken where both earth and moon are below sunshade
        self.ffis_below_sunshade = {}

        # dictionary mapping ffi nums's to all info about that ffi
        self.data_dic = {}

        # # debugging scaling of pixels
        # self.maxim = 0
        # self.minim = 1000000
    
    
    def get_arr(self, fits_filename, fits_folder_path):
        '''
        input: the name of the fits file we need to get info out of
                the path to the fodler where the fits file is in
        output: numpy array of four ccd images
        '''
#         fits.info(fits_folder_paths + fits_filename)
        numpy_arr = fits.getdata(fits_folder_path + fits_filename, ext=0)
        return numpy_arr

    def show_img(self, arr, title='', vmin=0, vmax=1):
        fig,ax = plt.subplots()
        im = ax.imshow(arr, cmap="gray", vmin=vmin, vmax=vmax)
        plt.grid(visible=False)
        colorbar = fig.colorbar(im)
        ax.set_title(title)
        plt.show()
        plt.close()
    
    def process_angles(self, save=True):
        '''
        input: None
        output: None
        iterates over each folder, and iterates through the file line by line and creates a
        dictionary with key=image number and value=tuple of values.
        It then pickles the dictionary and saves it into the angle 
        folder with the name "angles_data.pkl"
        '''
        # angles_dic = {}
        # order of items in Roland's files:
        #FIN ED MD Eel Eaz Mel Maz E1el E1az E2el E2az E3ez E3az E4el E4az M1el M1az M2el M2az M3el M3az M4el M4az

        # opens file and reads line by line
        for file_path in self.raw_angles_file_paths:
            orbit = file_path[36:38] if file_path[38] == '_' else file_path[36:37]
            with open(file_path, 'r') as file:
                for line in file.read().split('\n')[1:]:
                    arr = line.strip().split()
                    if len(arr)<2: break
                    arr = [float(arr[i]) if i>0 else str(arr[i]) for i in range(len(arr))]
                    ffi = arr[0]

                    # fucntion that edits values and rounds to 5 digits
                    deg_to_rad = lambda x: round(x*np.pi/180, 5)
                    dist_scaled_inverse = lambda x: round(1/(x/50), 5)
                    dist_scaled_inverse_sqrd = lambda x: round(1/(x/50)**2, 5)

                    # creates dictionary of values for each ffi
                    data_dic = {}
                    data_dic['ffi'] = ffi
                    data_dic['orbit'] = str(orbit)
                    data_dic['1/ED'] = dist_scaled_inverse(arr[1])
                    data_dic['1/MD'] = dist_scaled_inverse(arr[2])
                    data_dic['1/ED^2'] = dist_scaled_inverse_sqrd(arr[1])
                    data_dic['1/MD^2'] = dist_scaled_inverse_sqrd(arr[2])
                    data_dic['Eel'] = deg_to_rad(arr[3])
                    data_dic['Eaz'] = deg_to_rad(arr[4])
                    data_dic['Mel'] = deg_to_rad(arr[5])
                    data_dic['Maz'] = deg_to_rad(arr[6])
                    data_dic['E3el'] = deg_to_rad(arr[11])
                    data_dic['E3az'] = deg_to_rad(arr[12])
                    data_dic['M3el'] = deg_to_rad(arr[19])
                    data_dic['M3az'] = deg_to_rad(arr[20])
                    data_dic['below_sunshade'] = arr[3]<-5 and arr[5]<-5

                    # saves dictionary of dictionaries to the class
                    self.data_dic[ffi] = data_dic

        print(f"Dataset size: {len(self.data_dic)}")
        # saves dictionary to computer
        if save: 
            with open(self.angle_folder + self.angle_file, 'wb') as file:
                pickle.dump(self.data_dic, file)
                print(f"Saved angle file to: {str(file)}")
        print('finished processing angles\n')

    def process_save_images(self):
        '''
        input: None
        output: None
        Processes and saves all images.
        Pipeline:
            -Reads fits file
            -Removes rows and columns of black space in order to get a 4096x4096 image
            -Subtracts average background image (reference image) 
            -Median filter without downsampling
        '''
        
        rows_to_delete = range(2048, 2108)
        columns_to_delete = list(range(0, 44)) + list(range(2092, 2180)) + list(range(4228, 4272))
        for folder_path in self.fits_folder_paths:
            orbit = folder_path[27:29] if folder_path[29]=='_' else folder_path[27:28]
            
            # iterates through all original image files in orbit directory
            pbar_files = tqdm(os.listdir(folder_path))
            for fits_filename in pbar_files:
                pbar_files.set_description(f"orbit {orbit} images")

                # skips files that we do not want - ex: not camera 3
                if not (len(fits_filename) > 40 and fits_filename[-7:]=='fits.gz' and fits_filename[27] == '3'): continue

                ffi_num = fits_filename[18:26]
                arr = self.get_arr(fits_filename, folder_path)

                # self.show_img(arr, title=f'Unprocessed image {ffi_num} O{orbit}', vmin=0, vmax=633118)

                # removes black bars
                arr = np.delete(arr, rows_to_delete, axis=0)
                arr = np.delete(arr, columns_to_delete, axis=1)

                # self.show_img(arr, title=f'Removed bars {ffi_num} O{orbit}', vmin=0, vmax=633118)

                # SUBTRACTING background image stage
                # finds the orbit's average image to subtract
                # neither 57 nor 58 have images where both E + M below sunshade
                orbit_to_subtract = orbit
                if orbit == '10': orbit_to_subtract = '9'
                if orbit == '11': orbit_to_subtract = '12'
                if orbit == '13': orbit_to_subtract = '14'
                if orbit == '15': orbit_to_subtract = '16'
                if orbit == '28': orbit_to_subtract = '27'
                if orbit == '30': orbit_to_subtract = '29'
                if orbit == '32': orbit_to_subtract = '31'
                if orbit == '34': orbit_to_subtract = '33'
                if orbit == '37': orbit_to_subtract = '38'
                if orbit == '56': orbit_to_subtract = '55'
                if orbit == '60': orbit_to_subtract = '59'
                if orbit == '61': orbit_to_subtract = '62'

                # find image and subtract it
                try:
                    # found image, and subtract it
                    background_avg_image = pickle.load(open(background_ccd_folder+'O'+orbit_to_subtract+'_background_ccd.pkl', "rb"))
                    arr = arr - background_avg_image
                except:
                    # could not find background image, meaning no image where E+M both below sunshade exists
                    print('orbit', orbit, 'does not have any images where E + M below sunshade, skipping subtraction in dataset')
                    # return

                # self.show_img(arr, title=f'Subtracted background {ffi_num} O{orbit}', vmin=0, vmax=633118)

                # Median filter (without downsampling)
                arr = medfilt2d(arr, kernel_size=7)

                # self.show_img(arr, title=f'Median-filtered {ffi_num} O{orbit}', vmin=0, vmax=633118)
                
                # pickle and save
                with open(self.ccd_folder + fits_filename[:-8] + '_processed_im4096x4096.pkl', 'wb') as file:
                    pickle.dump(arr, file)
                    # print(f"image saved, ffi {ffi_num}, O{orbit}")

        print('Images done.')


    def save_original_image_displays(self):
        '''
        input: None
        output: None
        saves pdf of each orbit's images in two ~30x30 files, where each 'box' contains an image. One
            ~30x30 file contains all original images, and the other 30x30 file contains all of the
            processed images

        **processed images must be already saved to "ccds_background_subtracted_im4096x4096" folder in order to
            be saved to the processed image display files**

        **this code works for camera 3 now, can be edited in the future to be more general**
        '''
        # # angle data for all datapoints (3737604 points total)
        # data_angles = pickle.load(open(angle_folder+'angles_O9-64_data_dic.pkl', "rb"))

        # dictionary of orbits to list of all ffi numbers in that orbit
        orbit_to_ffis = {}
        for orbit_num in range(9, 65):
            orbit_to_ffis[str(orbit_num)] = []
        for ffi, data_dic in self.data_dic.items():
            orbit_to_ffis[data_dic['orbit']].append(ffi)

        # directories that contain separate orbits' images
        fits_orbit_folders = []
        for i in range(9, 65):
            fits_orbit_folders.append("/pdo/users/roland/SL_data/O" + str(i) + "_data/")

        # dictionary mapping ffi number to processed image array.
        # This is populated by iterating through all pkl files in the ccd_folder
        ffi_to_processed_image = {}
        for pkl_filename in os.listdir(ccd_folder):
            ffi_num = pkl_filename[18:26]
            ffi_to_processed_image[ffi_num] = pickle.load(open(ccd_folder+pkl_filename, "rb"))

        # this iterates through all orbit directories and makes a display file every iteration
        counter = 0
        rows_to_delete = range(2048, 2108)
        columns_to_delete = list(range(0, 44)) + list(range(2092, 2180)) + list(range(4228, 4272))
        for folder_path in fits_orbit_folders:

            orbit = folder_path[27:29] if folder_path[29]=='_' else folder_path[27:28] 
            ffi_to_original_image = {}

            # iterates through all original image files in orbit directory
            for fits_filename in os.listdir(folder_path):
                if len(fits_filename) > 40 and fits_filename[-7:]=='fits.gz' and fits_filename[27] == '3':
                    # gathers array image and ffi number for that image
                    arr = self.get_arr(fits_filename, folder_path)
                    ffi = fits_filename[18:26]

                    # # semi-arbitrary clip in order to see the stars/light scatter better
                    # arr = np.clip(arr, 0, 633118)

                    # removes black bars
                    arr = np.delete(arr, rows_to_delete, axis=0)
                    arr = np.delete(arr, columns_to_delete, axis=1)

                    # populates dictionary mapping for ffi_to_original_image
                    ffi_to_original_image[ffi] = arr
                    print('O' + self.data_dic[ffi]['orbit'], counter, fits_filename)
                    counter += 1
                else:
                    # for files not saved (ex: not camera 3)
                    print('Skipped' + fits_filename)

            # initializes figure for displaying images. Will be a square. Need 2*<size of dict>
            # because both original and processed images are going in the same display
            rows = math.ceil((2*len(ffi_to_original_image))**0.5)
            if rows%2 != 0:
                rows += 1

            fig, axs = plt.subplots(rows, rows, figsize=(15, 15))
            fig.suptitle('Orbit ' + orbit, fontsize = 25)
            print(f"Orbit {orbit} has {rows} rows")

            # iterates through all images & ffi numbers in orbit and gives it a place in the figure
            idx = 0 # keeps track of image position in the figure
            for ffi in sorted(ffi_to_original_image.keys()):
                # create title
                data_rounded = {key: round(float(val), 2) for key, val in self.data_dic[ffi].items()}
                data_line1 = str({'1/ED':data_rounded['1/ED'], '1/MD':data_rounded['1/MD'], '1/ED^2':data_rounded['1/ED^2'], '1/MD^2':data_rounded['1/MD^2']})
                data_line2 = str({'Eel':data_rounded['Eel'], 'Eaz':data_rounded['Eaz'], 'Mel':data_rounded['Mel'], 'Maz':data_rounded['Maz']})
                data_line3 = str({'E3el':data_rounded['E3el'], 'E3az':data_rounded['E3az'], 'M3el':data_rounded['M3el'], 'M3az':data_rounded['M3az']})
                plottitle = ffi + '\n' + data_line1 + '\n' + data_line2 + '\n' + data_line3
                
                # for original image
                # the min and max values are arbitrarily set in order to see the stars better
                axs[idx//rows, idx%rows].imshow(ffi_to_original_image[ffi], cmap="gray", vmin=0, vmax=633118)
                axs[idx//rows, idx%rows].axis('off')
                axs[idx//rows, idx%rows].set_title(plottitle, fontsize=1.2, pad=1, loc='left')
                idx += 1

                # for processed image
                # the min and max values are set to 0 and 1, which are the min and max pixel values
                axs[idx//rows, idx%rows].imshow(ffi_to_processed_image[ffi], cmap="gray", vmin=0, vmax=1)
                axs[idx//rows, idx%rows].axis('off')
                idx += 1

            # removes axes for the rest of the images
            for i in range(idx, rows**2):
                axs[i//rows, i%rows].axis('off')

            # shows image and saves it
            plt.show()
            #
            #
            # MAY NEED TO INCREASE DPI
            #
            #
            fig.savefig(os.path.join(self.display_images_folder, f"O{orbit}_images.pdf"), dpi=4096)
            plt.close()
            print('Image display for O' + orbit + ' done!')
        print('Image displays done.')


    def save_background_ccds(self):

        # this iterates through all orbit directories and makes a display file every iteration
        rows_to_delete = range(2048, 2108)
        columns_to_delete = list(range(0, 44)) + list(range(2092, 2180)) + list(range(4228, 4272))
        for folder_path in self.fits_folder_paths:

            orbit = folder_path[27:29] if folder_path[29]=='_' else folder_path[27:28] 
            orbit_images_below_sunshade = []

            # iterates through all original image files in orbit directory

            pbar_files = tqdm(os.listdir(folder_path))
            for fits_filename in pbar_files:
                pbar_files.set_description(f"orbit {orbit} images")
                if len(fits_filename) > 40 and fits_filename[-7:]=='fits.gz' and fits_filename[27] == '3':

                    ffi_num = fits_filename[18:26]
                    # ignore cases where Earth or Moon are above sunshade
                    if not self.data_dic[ffi_num]['below_sunshade']: continue
            
                    # gathers array image and ffi number for that image
                    arr = self.get_arr(fits_filename, folder_path)

                    # removes black bars
                    arr = np.delete(arr, rows_to_delete, axis=0)
                    arr = np.delete(arr, columns_to_delete, axis=1)

                    # adds to collection of orbit's images below sunshade
                    orbit_images_below_sunshade.append(arr)
                else:
                    # for files not saved (ex: not camera 3)
                    # print('Skipped' + fits_filename)
                    pass

            
            # in the case that no images are below sunshade
            if len(orbit_images_below_sunshade)==0:
                print(f'No images below sunshade in orbit {orbit}')
                continue

            # averages images
            arr_average = np.average(np.array(orbit_images_below_sunshade), axis = 0)
            
            # pickles and saves image
            with open(self.background_ccd_folder + 'O' + orbit + '_background_ccd.pkl', 'wb') as file:      
                pickle.dump(arr_average, file)
                print(f"Saved average background image to {str(file)}")

            print(f"{len(orbit_images_below_sunshade)} images averaged in orbit {orbit}")

            # displays image for each orbit
            fig,ax = plt.subplots()
            im = ax.imshow(arr_average, cmap="gray", vmin=0, vmax=633118) # for when all pixels are between 0 and 1
            plt.grid(visible=False)
            colorbar = fig.colorbar(im)
            plt.title(f"Average Orbit {orbit} image below sunshade")
            plt.show()
            plt.close()



    
    
    def run(self):
        '''
        input: None
        output: None
        runs the functionality described above using the functions within this class
        '''
        # ANGLES (MUST ALWAYS RUN)
        # processes angles
        self.process_angles(save=True)

        # # BACKGROUND IMAGES
        # self.save_background_ccds()


        # IMAGES
        # processes and saves fits files, images
        self.process_save_images()

        return
        
        
        # # DISPLAYS
        # # makes and saves dataset image displays
        # self.save_original_image_displays()
        
    
        
        # # IMAGES
        # # processes and saves fits files, images

        # # data_angles = pickle.load(open(angle_folder+angle_file, "rb"))
        
        # counter = 0
        # for folder_path in self.fits_folder_paths:
        #     for fits_filename in os.listdir(folder_path):
        #         if len(fits_filename) > 40 and fits_filename[-7:]=='fits.gz' and fits_filename[27] == '3':
        #             # print(fits_filename, fits_filename[-4:], len(fits_filename))
        #             arr = self.get_arr(fits_filename, folder_path)
        #             ffi = fits_filename[18:26]

        #             self.save_process_CCD(arr, 16, 10, fits_filename, ffi)
        #             # print(counter, f"O{self.data_dic[ffi]['orbit']}", fits_filename)
        #             counter += 1
        #         else:
        #             print('Skipped' + fits_filename)
        # print('Images done.')

In [32]:
# Filepaths

fits_folder_paths = []
raw_angles_file_paths = []
angle_folder = "/pdo/users/jlupoiii/TESS/data/angles/"
ccd_folder = "/pdo/users/jlupoiii/TESS/data/processed_images_im4096x4096/"
background_ccd_folder = "/pdo/users/jlupoiii/TESS/data/background_avg_ccds_im4096x4096/"
display_images_folder = "/pdo/users/jlupoiii/TESS/data/display_images_im4096x4096/"
angle_file = "angles_O9-64_data_dic.pkl"

for i in range(9, 64+1):
# for i in [13, 14, 19, 20]:
    # if i==9 or i==10 or i==61 or i==62 or i==63 or i==64: continue

    fits_folder_paths.append("/pdo/users/roland/SL_data/O" + str(i) + "_data/")
    raw_angles_file_paths.append("/pdo/users/roland/SL_data/altazzes/O" + str(i) + "_altaz.out")

In [ ]:
# Run 
processor_save_data = Preprocessing(fits_folder_paths, angle_folder, ccd_folder, background_ccd_folder, raw_angles_file_paths, angle_file, display_images_folder)
processor_save_data.run()

Dataset size: 37604
Saved angle file to: <_io.BufferedWriter name='/pdo/users/jlupoiii/TESS/data/angles/angles_O9-64_data_dic.pkl'>
finished processing angles



orbit 9 images:   4%|█                       | 28/634 [05:51<2:08:35, 12.73s/it]